# Libraries

In [14]:
# !pip install imbalanced-learn
# !pip install scikit-image

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/opt/tljh/user/bin/python3.9 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 65.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.6/195.6 KB 76.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 101.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 109.0 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the '/opt/tljh/user/bin/

In [16]:
from wettbewerb import load_references, save_predictions
from preprocess import *
from sklearn import preprocessing
import numpy as np
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split

In [21]:
# import zipfile
# with zipfile.ZipFile("training.zip", 'r') as zip_ref:
#     zip_ref.extractall("")

# Data import

In [30]:
ecg_leads, ecg_labels, fs, ecg_names = load_references(folder='training')

6000	 Dateien wurden geladen.


# Preprocessing

In [31]:
ecg_leads_edited = ecg_leads

### Noise removal (fan_multiscaled_2018)

In [32]:
filtered = []
for j, data in enumerate(ecg_leads_edited):
    data_ftt, freq = ecg_furier(ecg_leads_edited[j], fs)
    lowpassed = ecg_denoise_spectrum(data_ftt, freq, 0, 60)
    filtered.append(ecg_invfurier(lowpassed))

ecg_leads_edited = filtered

### Z-normalization

In [33]:
filtered = []
for j, data in enumerate(ecg_leads_edited):
    filtered.append(ecg_norm(ecg_leads_edited[j]))

ecg_leads_edited = filtered

### Length normalization (hsieh_detection_2020)

In [34]:
def ecg_split(data, limit, label):
    splitted = []
    ratio = len(data)/limit

    # If data is longer than limit, split with at least 50% overlap
    if ratio > 1:
        for i in range(0, int(np.ceil(2*ratio))):
            if i == int(np.ceil(2*ratio))-1:
                string = data[len(data)-limit:len(data)]
            else:
                string = data[int(np.floor(i*limit/2)):int(np.floor(i*limit/2+limit))]
            splitted.append(string)
        splitted = [x for x in splitted if x.shape[0] >= limit] # remove string which are shorter
        labels_multiplied = [label] * len(splitted)

    # If data is shorter than limit, add from the beginning
    elif ratio < 1:
        if ratio <= 0.5:
            data = np.tile(data, int(np.floor(1/ratio)))
        diff = limit - len(data)
        appended_data = data[0:diff]
        data = np.append(data, appended_data)
        splitted.append(data)
        labels_multiplied = [label] * 1

    # If it is the exact length, then don't alter it
    elif ratio == 1:
        splitted.append(data)
        labels_multiplied = [label] * 1
    return splitted, labels_multiplied

In [35]:
normalized = []
normalized_label = []
duration = 20
split = duration*fs
for j, data in enumerate(ecg_leads_edited):
    ecg_split_data, lab_mul = ecg_split(data, split, ecg_labels[j])
    normalized.extend(ecg_split_data)
    normalized_label.extend(lab_mul)

ecg_leads_edited = normalized
ecg_labels_edited = normalized_label

### Split training, validation, test data

In [41]:
# Validation set
X_train, X_test, y_train, y_test = train_test_split(ecg_leads_edited, ecg_labels_edited, test_size=0.2, shuffle = True, random_state = 8)

In [49]:
X_train = np.asarray(X_train)
y_train = np.asarray(y_train)

### Random oversampling for imbalanced training data

In [60]:
ros = RandomOverSampler(random_state=0, sampling_strategy='minority')
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

# Converts sublists to arrays
xfiltered = []
yfiltered = []
for j, data in enumerate(X_train):
    xfiltered.append(np.asarray(X_resampled[j]))
    yfiltered.append(np.asarray(y_resampled[j]))
X_train = xfiltered
y_train = yfiltered

### Label encoding (0: A, 1: N, 2: O, 3: ~)

In [ ]:
# get indices for each label
# idx_N = [i for i in range(len(ecg_labels_edited)) if ecg_labels_edited[i] == 'N']
# idx_A = [i for i in range(len(ecg_labels_edited)) if ecg_labels_edited[i] == 'A']
# idx_tilde = [i for i in range(len(ecg_labels_edited)) if ecg_labels_edited[i] == '~']
# idx_O = [i for i in range(len(ecg_labels_edited)) if ecg_labels_edited[i] == 'O']

In [ ]:
le = preprocessing.LabelEncoder()
le.fit(ecg_labels_edited)
ecg_labels_enc = le.transform(ecg_labels_edited)

In [ ]:
# d = []
# for p, data in enumerate(ecg_labels_edited):
#     d.append(
#         {
#             'ecg_data': ecg_names[p],
#             'label': ecg_labels_enc[p]
#         }
#     )
# features_names = pd.DataFrame(d)

# Classification (CNN)

## Neuronales Netz aufstellen
- [ ] nur auf Trainingsdaten
- [ ] Tuningprozess: [08 - Data Science & Machine Learning](:/67056066e2104708bcc4c6d27239e051)
- [ ] Tuning der Parameter: https://keras.io/guides/keras_tuner/getting_started/
- [ ] Auch ROC, Confusion Matrix und Learning Curves ausleiten... (TP, TN, ...)
- [ ] Schnitttechnik der Daten (nur R-Peaks cutten)
- [ ] Auf Supercomputer auslegen (Anleitung s. Moodle)
- [ ] weitere Trainingsdaten durchgehen (s. andere)